In [113]:
# load data
from glob import glob 
raw_data = glob("/Users/yeonghwanchoi/Documents/git/EDA/EDA_consumtion trend of government subsidies/data/*")
raw_data[0]

'/Users/yeonghwanchoi/Documents/git/EDA/EDA_consumtion trend of government subsidies/data/KRI-DAC_Jeju_data6.txt'

In [114]:
# conncat data
from glob import glob 
concat_list=[]
for i in range(0,len(raw_data)):
    df=pd.read_csv(raw_data[i])
    df["월"]=raw_data[i].split(".")[0][-1:]
    concat_list.append(df)

In [115]:
df=pd.concat(concat_list)
df

,OBJECTID,Field1,YM,SIDO,SIGUNGU,FranClass,Type,Time,TotalSpent,DisSpent,NumofSpent,NumofDisSpent,POINT_X,POINT_Y,월,X,Y
0,1,1,202006,제주특별자치도,제주시,영세,일반한식,00시,502000,0,10,0,877005.9834,1.479766e+06,6,NaN,NaN
1,2,2,202006,제주특별자치도,제주시,영세,단란주점,00시,1520000,0,8,0,877005.7447,1.479816e+06,6,NaN,NaN
2,3,3,202006,제주특별자치도,제주시,중소1,편의점,00시,482310,0,35,0,877056.6756,1.479616e+06,6,NaN,NaN
3,4,4,202006,제주특별자치도,제주시,영세,편의점,00시,38050,5450,3,1,877055.9593,1.479766e+06,6,NaN,NaN
4,5,5,202006,제주특별자치도,제주시,영세,일반한식,00시,32000,32000,1,1,877055.4817,1.479866e+06,6,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
280080,280081,280081,202008,제주특별자치도,제주시,영세,편의점,x시,13700,0,2,0,950266.5976,1.502458e+06,8,NaN,NaN
280081,280082,280082,202008,제주특별자치도,제주시,영세,일반한식,x시,56000,0,1,0,950317.5379,1.502258e+06,8,NaN,NaN
280082,280083,280083,202008,제주특별자치도,제주시,영세,일반한식,x시,28000,0,1,0,950469.1585,1.501909e+06,8,NaN,NaN
280083,280084,280084,202008,제주특별자치도,제주시,중소1,중국음식,x시,46000,0,3,0,950574.6388,1.500760e+06,8,NaN,NaN


In [18]:
# montly_spend = 

In [19]:
#dropped useless columns 
df.columns=df.columns.str.lower()
df.drop(['sido','x',"objectid",'y'],axis = 1, inplace=True)

In [ ]:
#rename koren named columns to eng
  

In [33]:
from pyproj import Proj 
proj_itrf = Proj(init='epsg:5179')
proj_wgs84 = Proj(init='epsg:4326')

In [36]:
df["lon"],df["lat"] = transform(proj_itrf,proj_wgs84,df['point_x'],df["point_y"])

In [37]:
df["lon_lat"]=df["lon"].astype(str)+'_'+df["lat"].astype(str)
df["lon_lat"].value_counts()

126.52427851055405_33.51290574138535     1559
126.52935695016222_33.49899169591379     1242
126.52880376641139_33.499886709171996    1208
126.47793511858133_33.48574932701181     1193
126.58018359519565_33.516714007010115    1051
                                         ... 
126.30425990442274_33.287409183817225       1
126.69051885754698_33.51612507132969        1
126.45625055766716_33.46474533801859        1
126.21819468884902_33.24383319689915        1
126.35828653698235_33.468897035464025       1
Name: lon_lat, Length: 15369, dtype: int64

In [117]:
#import rename module  
import type_names
input_data=type_names.rename()
for key, value in input_data:
    df["Type"].replace(dict.fromkeys(key, value), inplace=True)

In [118]:
for key, value in input_data:
    df["Type"].replace(dict.fromkeys(key, value), inplace=True)
df

,OBJECTID,Field1,YM,SIDO,SIGUNGU,FranClass,Type,Time,TotalSpent,DisSpent,NumofSpent,NumofDisSpent,POINT_X,POINT_Y,월,X,Y
0,1,1,202006,제주특별자치도,제주시,영세,외식,00시,502000,0,10,0,877005.9834,1.479766e+06,6,NaN,NaN
1,2,2,202006,제주특별자치도,제주시,영세,유흥/주점,00시,1520000,0,8,0,877005.7447,1.479816e+06,6,NaN,NaN
2,3,3,202006,제주특별자치도,제주시,중소1,식료품,00시,482310,0,35,0,877056.6756,1.479616e+06,6,NaN,NaN
3,4,4,202006,제주특별자치도,제주시,영세,식료품,00시,38050,5450,3,1,877055.9593,1.479766e+06,6,NaN,NaN
4,5,5,202006,제주특별자치도,제주시,영세,외식,00시,32000,32000,1,1,877055.4817,1.479866e+06,6,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
280080,280081,280081,202008,제주특별자치도,제주시,영세,식료품,x시,13700,0,2,0,950266.5976,1.502458e+06,8,NaN,NaN
280081,280082,280082,202008,제주특별자치도,제주시,영세,외식,x시,56000,0,1,0,950317.5379,1.502258e+06,8,NaN,NaN
280082,280083,280083,202008,제주특별자치도,제주시,영세,외식,x시,28000,0,1,0,950469.1585,1.501909e+06,8,NaN,NaN
280083,280084,280084,202008,제주특별자치도,제주시,중소1,외식,x시,46000,0,3,0,950574.6388,1.500760e+06,8,NaN,NaN
